# Bonus Homework 5:  Fair Machine Learning

The purpose of the data set is to compare classification of a mixed population (with a mixture of Gaussians distribution) with access and without access to the group attribute as worked out analytically on homework 5.

The data was generated using the same assumptions used in written  homework 5 (problems 2 and 3). Completing  this notebook should help you verify your analytic calculations.

When comparing results, take into account that this example here uses  only use a finite data sample (5,000 samples).  The empirically estimated rates should be close to the theoretical ones, but not identical.

## Preliminaries

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot  as plt 

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

%matplotlib inline

## Prepare Data

<div class="alert alert-block alert-info"> Problem 0.1 </div>
Read the file 'fair_ml_data.csv' provided with this homework set into a panda's data frame

In [2]:
data_dir="../data/fair_ml"


In [3]:
filename=data_dir+"/fair_ml_data.csv"

In [4]:
data_all=pd.read_csv(filename)
data_all.head(10)

,A,Y,X
0,0,0,0.146392
1,0,0,-0.477548
2,0,0,-0.061700
3,0,1,-0.060675
4,0,1,0.029771
5,1,1,-0.769145
6,0,1,-0.198145
7,0,0,-0.961251
8,0,1,0.209669
9,0,1,0.374967


<div class="alert alert-block alert-info"> Problem 0.2 </div>
Split data in a 50% training  and 50% valuation sets

In [5]:
data_train,data_val=train_test_split(data_all,test_size=0.5)
data_train.shape,data_val.shape

((2500, 3), (2500, 3))

<div class="alert alert-block alert-info"> Problem 0.3 </div>

In this problem set you will need to fit LDA models to 
1. the majority group $a=0$ 
2. minority group $a=1$
3. to the whole data set irrespective of group affiliation. 

Prepare `numpy` arrays for training and validation so  that you can fit those three models:
1. Arrays `X0_train`,`X0_val`,`Y0_train`,`Y0_val` for members of the majority group `a=0`.
2. Arrays `X1_train`,`X1_val`,`Y1_train`,`Y1_val` for members of the minority group `a=1`.
3. Arrays `X_train`,`X_val`,`Y_train`,`Y_val` for all members irrespective of group affiliation.


<div class="alert alert-block alert-success">  Majority Group Data $a=1$</div>

In [8]:
data0_train=data_train[data_train["A"]==0]
data0_train.head(10)

,A,Y,X
3023,0,0,-1.461510
1326,0,1,0.986742
368,0,0,-1.040235
4810,0,0,-0.183249
2444,0,1,1.140546
543,0,1,0.057791
4492,0,0,-0.550809
3521,0,0,-1.142724
1863,0,1,0.221725
1289,0,0,0.086744


In [9]:
X0_train=data0_train[["X"]].values
Y0_train=data0_train[["Y"]].values.ravel()
X0_train.shape,Y0_train.shape

((2005, 1), (2005,))

In [10]:

data0_val=data_val[data_val["A"]==0]
data0_val.head(10)

,A,Y,X
3553,0,0,-0.349195
3357,0,1,0.693833
4709,0,0,0.379126
2302,0,0,-0.987815
1637,0,0,-1.313846
2213,0,1,0.363108
3291,0,0,-0.246370
269,0,0,-0.476348
1094,0,1,0.228456
1729,0,1,1.027536


In [11]:
X0_val=data0_val[["X"]].values
Y0_val=data0_val[["Y"]].values.ravel()
X0_val.shape,Y0_val.shape

((2010, 1), (2010,))

<div class="alert alert-block alert-success">  Minority Group Data $a=0$</div>

In [12]:
data1_train=data_train[data_train["A"]==1]
data1_train.head(10)

,A,Y,X
2797,1,1,-1.714137
1875,1,1,-0.637611
3194,1,1,-1.583844
4960,1,1,-1.750449
736,1,1,-0.678825
2335,1,0,1.680634
261,1,1,-0.959513
4939,1,0,1.432571
2637,1,1,-0.939767
2058,1,1,-0.669967


In [13]:
X1_train=data1_train[["X"]].values
Y1_train=data1_train[["Y"]].values.ravel()
X1_train.shape,Y1_train.shape

((495, 1), (495,))

In [14]:
data1_val=data_val[data_val["A"]==1]
data1_val.head(10)

,A,Y,X
3898,1,0,-0.638472
4759,1,1,-1.948657
2435,1,0,0.711707
1890,1,1,-1.616460
3174,1,1,-0.709210
4898,1,1,-1.655591
4145,1,1,-1.335096
1520,1,0,1.033209
3301,1,1,-0.604999
2884,1,1,-1.420092


In [15]:
X1_val=data1_val[["X"]].values
Y1_val=data1_val[["Y"]].values.ravel()
X1_val.shape,Y1_val.shape

((490, 1), (490,))

<div class="alert alert-block alert-success">  All members  Data</div>

In [6]:
X_train=data_train[["X"]].values
Y_train=data_train[["Y"]].values.ravel()

In [7]:
X_val=data_val[["X"]].values
Y_val=data_val[["Y"]].values.ravel()

##  Per Group Classifiers

### Majority Group model

<div class="alert alert-block alert-info"> Problem 1.0 </div>
Fit an LDA model to the training examples where $a=1$

In [16]:
model0=LDA()
model0.fit(X0_train,Y0_train)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

<div class="alert alert-block alert-info"> Problem 1.1 </div>
Compute the model accuracy on the majority population $a=0$.

Because of finite sample (we only have N=2,000 samples) you should get results close, but not identical to what you derived on the written homework.

In [17]:
Y0_pred=model0.predict(X0_val)
np.mean(Y0_pred==Y0_val)

0.8378109452736319

<div class="alert alert-block alert-info"> Problem 1.2 </div>
Estimate  the rate of true positives and false negatives on the majority population $a=0$ using the validation data
you set aside before.

[HINT: The function `sklear.metrix.confusion_matrix may be useful]

In [18]:
cm0=metrics.confusion_matrix(Y0_val,Y0_pred)
cm0

array([[982, 162],
       [164, 702]], dtype=int64)

In [19]:
TN0=cm0[0,0]
TP0=cm0[1,1]
FP0=cm0[0,1]
FN0=cm0[1,0]

In [20]:
tpr0= TP0/(TP0+FN0)
tpr0

0.8106235565819861

In [21]:
fpr0= FP0/(FP0+TN0)
fpr0

0.14160839160839161

### Minority Group Model

<div class="alert alert-block alert-info"> Problem 2.0 </div>
Fit an LDA model to the training examples where $a=0$

In [22]:
model1=LDA()
model1.fit(X1_train,Y1_train)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

<div class="alert alert-block alert-info"> Problem 2.1 </div>
Compute the model accuracy on the majority population $a=1$.

In [23]:
Y1_pred=model1.predict(X1_val)
np.mean(Y1_pred==Y1_val)

0.9877551020408163

<div class="alert alert-block alert-info"> Problem 2.2 </div>
Estimate  the rate of true positives and false negatives on the minority population $a=1$ using the validation data
you set aside before.

In [24]:
cm1=metrics.confusion_matrix(Y1_val,Y1_pred)
cm1

array([[121,   3],
       [  3, 363]], dtype=int64)

In [25]:
TN1=cm1[0,0]
TP1=cm1[1,1]
FP1=cm1[0,1]
FN1=cm1[1,0]

In [26]:
tpr1= TP1/(TP1+FN1)
tpr1

0.9918032786885246

In [27]:
fpr1= FP1/(FP1+TN1)
fpr1

0.024193548387096774

<div class="alert alert-block alert-info"> Problem 2.3 </div>
Calculate the accuracy on the overall population (including $a=0,1$) if we use different classifiers for $a=0$ and $a=1$

In [28]:
(np.sum(Y0_pred==Y0_val)+np.sum(Y1_pred==Y1_val))/len(Y_val)

0.8672

## Fair (Anti-Classification) Model

<div class="alert alert-block alert-info"> Problem 3.1 </div>
Train an LDA model using the whole dataset to predict $y$ based on $x$ without making use of the $a$ attribute.

In [29]:
base_model=LDA()

In [30]:
base_model.fit(X_train,Y_train)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

<div class="alert alert-block alert-info"> Problem 3.1 </div>
Estimate the accuracy of the fair model

Accuracy of base model

In [31]:
Y_pred=base_model.predict(X_val)
np.mean(Y_pred==Y_val)

0.6536

<div class="alert alert-block alert-info"> Problem 3.2 </div>
Compute the accuracy the fair model when applied only to the majority population
where $a=0$

In [32]:
Y0_pred=base_model.predict(X0_val)
np.mean(Y0_pred==Y0_val)

0.800497512437811

In [33]:
cm0=metrics.confusion_matrix(Y0_val,Y0_pred)
cm0

array([[820, 324],
       [ 77, 789]], dtype=int64)

<div class="alert alert-block alert-info"> Problem 3.3 </div>
Compute true positive and false positive rates for the majority population  ($a=0$).

In [34]:
TN0=cm0[0,0]
TP0=cm0[1,1]
FP0=cm0[0,1]
FN0=cm0[1,0]

In [35]:
tpr0= TP0/(TP0+FN0)
tpr0

0.9110854503464203

In [36]:
fpr0= FP0/(FP0+TN0)
fpr0

0.28321678321678323

<div class="alert alert-block alert-info"> Problem 3.3 </div>
Compute the accuracy the fair model when applied only to the majority population
where $a=1$

In [37]:
Y1_pred=base_model.predict(X1_val)
np.mean(Y1_pred==Y1_val)

0.05102040816326531

<div class="alert alert-block alert-info"> Problem 3.4 </div>
Compute true positive and false positive rates for the minority population  ($a=1$).

In [38]:
cm1=metrics.confusion_matrix(Y1_val,Y1_pred)
cm1

array([[  2, 122],
       [343,  23]], dtype=int64)

In [39]:
TN1=cm1[0,0]
TP1=cm1[1,1]
FP1=cm1[0,1]
FN1=cm1[1,0]

In [40]:
tpr1= TP1/(TP1+FN1)
tpr1

0.06284153005464481

In [41]:
fpr1= FP1/(FP1+TN1)
fpr1

0.9838709677419355